In [8]:
# %load lotto.py
#!/usr/bin/python

import sys
from zip import unzip
from network import dl_file
from pprint import pprint

# variables
url = "http://www.bclc.com/documents/DownloadableNumbers/CSV/649.zip"
data_path = "data/"
file_name = "649.zip"

# download zip file and save
dl_file(url, data_path, file_name)

# unzip to csv
unzip(data_path, file_name)


### Downloading http://www.bclc.com/documents/DownloadableNumbers/CSV/649.zip and saving as data/649.zip


AttributeError: module 'urllib' has no attribute 'urlretrieve'

In [11]:
# %load network.py
#!/usr/bin/python

from urllib.request import urlretrieve

def dl_file(url, path, file):
  print("### Downloading " + url + " and saving as " + path + file)
  urllib.request.urlretrieve(url, path + file)
